In [1]:
import torch
import torch.nn as nn
import torchvision
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
num_epochs = 5
num_classes = 10
batch_size = 100 
learning_rate = 0.001

class Conv(nn.Module):
    def __init__(self, num_classes):
        super(Conv, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,16, kernel_size=5, stride = 1, padding = 2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16,32,kernel_size = 5, padding =2, stride =1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride =2))
        self.fc = nn.Linear(7*7*32, num_classes)
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
#         print(out.size(0))   #this is to check your implementation
        out = out.reshape(out.size(0), -1)#here we provide a batch of image so out.size(0) is the batch_size
        out = self.fc(out)
        return out

train_dataset = torchvision.datasets.MNIST(root='../ai61002s19/MNIST', train=True, transform = transforms.ToTensor(), download = False)
test_dataset = torchvision.datasets.MNIST(root='../ai61002s19/MNIST', train=False, transform = transforms.ToTensor(), download = False)
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)

model = Conv(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)
# out = model(train_dataset[0][0].reshape(1,1,28,28))#1-no. of images, 1-depth of the image, height, width

total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
#         print(images.shape)
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

model.eval()  #it is typically used for batchnorm so that it uses moving variance and mean instead of the whole batch
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Epoch [1/5], Step [100/600], Loss: 0.1840
Epoch [1/5], Step [200/600], Loss: 0.1319
Epoch [1/5], Step [300/600], Loss: 0.0985
Epoch [1/5], Step [400/600], Loss: 0.0702
Epoch [1/5], Step [500/600], Loss: 0.0764
Epoch [1/5], Step [600/600], Loss: 0.0713
Epoch [2/5], Step [100/600], Loss: 0.0245
Epoch [2/5], Step [200/600], Loss: 0.0640
Epoch [2/5], Step [300/600], Loss: 0.0906
Epoch [2/5], Step [400/600], Loss: 0.0248
Epoch [2/5], Step [500/600], Loss: 0.0114
Epoch [2/5], Step [600/600], Loss: 0.1116
Epoch [3/5], Step [100/600], Loss: 0.0559
Epoch [3/5], Step [200/600], Loss: 0.0767
Epoch [3/5], Step [300/600], Loss: 0.1913
Epoch [3/5], Step [400/600], Loss: 0.0453
Epoch [3/5], Step [500/600], Loss: 0.0302
Epoch [3/5], Step [600/600], Loss: 0.0527
Epoch [4/5], Step [100/600], Loss: 0.0102
Epoch [4/5], Step [200/600], Loss: 0.0629
Epoch [4/5], Step [300/600], Loss: 0.0215
Epoch [4/5], Step [400/600], Loss: 0.0314
Epoch [4/5], Step [500/600], Loss: 0.0040
Epoch [4/5], Step [600/600], Loss:

In [ ]:
#Do Practise after transfer learning and loading a custom dataset. 

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
batch_size = 100
num_epochs = 30
learning_rate = 0.001
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(), 
    transforms.RandomCrop(32), 
    transforms.ToTensor()])

train_dataset = torchvision.datasets.CIFAR10(root='../ai61002s19/', train=True, transform = transform, download = True)
test_dataset = torchvision.datasets.CIFAR10(root='../ai61002s19/', train=False, transform = transforms.ToTensor(), download = False)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size = batch_size, shuffle= True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size = batch_size, shuffle= False)

class ResidualBlock(nn.Module):
    def __init__(self, in_channel, out_channel,stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size = 3, stride = stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(out_channel, out_channel, kernel_size = 3, stride = 1, padding=1)  #we change the size only once
        self.bn2 = nn.BatchNorm2d(out_channel)
        self.downsample = downsample
    def forward(self,x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:#to be used when input size does not match output size
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return(out)

class ResNet(nn.Module):
    def __init__(self, block, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channel = 16
        self.conv1 = nn.Conv2d(3,16, stride =1, kernel_size = 3, padding = 1)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU()
        self.block1 = self.make_layer(block, 16, 1)
        self.block2 = self.make_layer(block, 16, 1)
        self.block3 = self.make_layer(block, 32, 2)
        self.block4 = self.make_layer(block, 32, 1)
        self.block5 = self.make_layer(block, 64, 2)
        self.block6 = self.make_layer(block, 64, 1)
        self.avg_pool = nn.AvgPool2d(8) #8 is the kernel size so it is taking average of 8x8
        self.fc = nn.Linear(64, num_classes)
    def make_layer(self, block, out_channel, stride=1):
        downsample = None
        if(stride!=1) or (self.in_channel != out_channel):#input size not equal to output size only when stride not 1 or input channel and output channel are not same 
            downsample = nn.Sequential(
            nn.Conv2d(self.in_channel, out_channel, kernel_size = 3, stride = stride, padding = 1),
            nn.BatchNorm2d(out_channel))
        out_layer = block(self.in_channel, out_channel, stride, downsample)
        self.in_channel = out_channel
        return(out_layer)
    def forward(self,x):
        out = self.conv1(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.block4(out)
        out = self.block5(out)
        out = self.block6(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
model = ResNet(ResidualBlock).to(device)
# print(train_dataset[0][0].shape)
# out = model(train_dataset[0][0].reshape(1,3,32,32))

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# Train the model
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)